In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
# import xgboost as xgb
import pickle
import plotly.graph_objects as go

In [4]:
monthly_data=pd.read_csv("Monthly_Training_Data.csv",usecols=["Service Name","Month","Year","Count"])
weekly_data=pd.read_csv("Weekly_training_data.csv",usecols=["Service Name","Month","Year","Count","wom"])
daily_data = pd.read_csv("daily_training_data.csv",usecols=["Service Name","Month","Year","Count","DayMonth"])
print(weekly_data.shape,monthly_data.shape, daily_data.shape)
daily_data.head()

(4148, 5) (1392, 4) (11784, 5)


,Service Name,Count,DayMonth,Month,Year
0,Examination Under Anathesia,1,2,1,2016
1,Femto-LASIK,7,2,1,2016
2,IOL Implantation,1,2,1,2016
3,Intravitreal Lucentis Injection,8,2,1,2016
4,Laser-Assisted in Situ Keratomileusis (LASIK),12,2,1,2016


In [49]:
# # monthly_data=monthly_data.sort_values(by=["Count"],ascending=False).iloc[0:100]
# monthly_data=monthly_data[(monthly_data['Year'] == 2018) & (monthly_data['Month'] == 12)]
# monthly_data=monthly_data.sort_values(by=["Count"],ascending=False).iloc[0:20]
# monthly_data

In [14]:
with open('operations_names_en_to_ar.pickle', 'rb') as f:
    operations_names_en_to_ar = pickle.load(f)

In [51]:
# Service_Names_to_index = {name : idx for idx, name in enumerate(weekly_data['Service Name'].unique())}
# Service_Index_to_name= {idx : name for idx, name in enumerate(weekly_data['Service Name'].unique())}
# monthly_Service_Index_to_name= {idx : name for idx, name in enumerate(weekly_data['Service Name'].unique())}

In [7]:
def encode_monthly_names(df, uniq):
    Service_Names_to_index = {name : idx for idx, name in enumerate(uniq)}
    df['Service Name']=df['Service Name'].map(Service_Names_to_index)
    return df

In [13]:
def decode_monthly_names(df, uniq):
    Service_Index_to_name= {idx : name for idx, name in enumerate(uniq)}
    df['Service Name']=df['Service Name'].map(Service_Index_to_name)
    return df

In [54]:
def encode_weekly_names(df):
    Service_Names_to_index = {name : idx for idx, name in enumerate(weekly_data['Service Name'].unique())}
    df['Service Name']=df['Service Name'].map(Service_Names_to_index)
    return df

In [55]:
def decode_weekly_names(df):
    Service_Index_to_name= {idx : name for idx, name in enumerate(weekly_data['Service Name'].unique())}
    df['Service Name']=df['Service Name'].map(Service_Index_to_name)
    return df

In [8]:
# weekly_processed_data = weekly_data.copy()
# weekly_processed_data=encode_weekly_names(weekly_processed_data)
# monthly_processed_data= monthly_data.copy()
# monthly_processed_data=encode_monthly_names(monthly_processed_data)
# monthly_processed_data.head()
# monthly_processed_data=encode_monthly_names(monthly_data)
uniq = daily_data['Service Name'].unique() 
daily_data = encode_monthly_names(daily_data, uniq)
daily_data.head()

,Service Name,Count,DayMonth,Month,Year
0,0,1,2,1,2016
1,1,7,2,1,2016
2,2,1,2,1,2016
3,3,8,2,1,2016
4,4,12,2,1,2016


In [57]:
# X=data.drop(["Count"],axis=1)
# y=data["Count"]

In [9]:
monthly_split_index = daily_data[(daily_data['Year'] == 2018) & (daily_data['Month'] == 6)].iloc[-1].name
# weekly_split_index = weekly_processed_data[(weekly_processed_data['Year'] == 2018) & (weekly_processed_data['Month'] == 11)].iloc[-1].name
print(monthly_split_index)

9697


In [10]:
# X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42)
def get_X_y(df,split_index):
    X_train = df.iloc[:split_index].drop('Count', axis=1)
    X_test = df.iloc[split_index:].drop('Count', axis=1)

    y_train = df.iloc[:split_index]['Count']
    y_test = df.iloc[split_index:]['Count']

    return X_train, X_test, y_train, y_test


In [12]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def element_mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return (np.abs((y_true - y_pred) / y_true)) * 100

In [61]:
# weekly_processed_data[(weekly_processed_data['Year'] == 2018) & (weekly_processed_data['Month'] == 12) & (weekly_processed_data['wom'] == 1) & (weekly_processed_data['wom'] == 2)]

In [62]:
def weekly_prediction_cases(regressor):
     week_ahead = weekly_processed_data[(weekly_processed_data['Year'] == 2018) & (weekly_processed_data['Month'] == 12) & (weekly_processed_data['wom'] == 1)]
     
     two_weeks_ahead = weekly_processed_data[(weekly_processed_data['Year'] == 2018) & (weekly_processed_data['Month'] == 12) & ((weekly_processed_data['wom'] == 1) | (weekly_processed_data['wom'] == 2))]
     

     week_ahead_pred = regressor.predict(week_ahead.drop('Count', axis=1))
     two_weeks_ahead_pred = regressor.predict(two_weeks_ahead.drop('Count', axis=1))
     # half_year_pred = regressor.predict(half_year.drop('Count', axis=1))
     
     err = mean_absolute_error(week_ahead['Count'], week_ahead_pred)
     week_ahead_MAPE = mean_absolute_percentage_error(week_ahead['Count'], week_ahead_pred)
     two_weeks_ahead_MAPE = mean_absolute_percentage_error( two_weeks_ahead['Count'], two_weeks_ahead_pred)
     # half_year_MAPE = mean_absolute_percentage_error(half_year['Count'], half_year_pred) 
     
     week_ahead["predictions"]=week_ahead_pred
     two_weeks_ahead["predictions"]=two_weeks_ahead_pred
     # half_year["predictions"]=half_year_pred
     week_ahead=decode_weekly_names(week_ahead)
     two_weeks_ahead=decode_weekly_names(two_weeks_ahead)
     # half_year['Service Name']=half_year['Service Name'].map(Service_Index_to_name)

     week_ahead['Service Name AR']=week_ahead['Service Name'].map(operations_names_en_to_ar)
     two_weeks_ahead['Service Name AR']=two_weeks_ahead['Service Name'].map(operations_names_en_to_ar)
     # half_year['Service Name AR']=half_year['Service Name'].map(operations_names_en_to_ar)     

     return err,week_ahead_MAPE, two_weeks_ahead_MAPE, week_ahead, two_weeks_ahead

In [71]:
def monthly_prediction_cases(regressor):
     month_ahead = monthly_processed_data[(monthly_processed_data['Year'] == 2018) & (monthly_processed_data['Month'] == 12)].sort_values(by=["Count"],ascending=False).iloc[0:10]
     
     quarter_year = monthly_processed_data[(monthly_processed_data['Year'] == 2018) & (monthly_processed_data['Month'] >= 9)]
     
     half_year = monthly_processed_data[(monthly_processed_data['Year'] == 2018) & (monthly_processed_data['Month'] >= 7)]

     month_ahead_pred = regressor.predict(month_ahead.drop('Count', axis=1))
     quarter_year_pred = regressor.predict(quarter_year.drop('Count', axis=1))
     half_year_pred = regressor.predict(half_year.drop('Count', axis=1))
     
     err = mean_absolute_error(month_ahead['Count'], month_ahead_pred)
     month_MAPE = mean_absolute_percentage_error(month_ahead['Count'], month_ahead_pred)
     quarter_year_MAPE = mean_absolute_percentage_error( quarter_year['Count'], quarter_year_pred)
     half_year_MAPE = mean_absolute_percentage_error(half_year['Count'], half_year_pred) 
     
     month_ahead["predictions"]=month_ahead_pred
     quarter_year["predictions"]=quarter_year_pred
     half_year["predictions"]=half_year_pred

     month_ahead=decode_monthly_names(month_ahead)
     quarter_year=decode_monthly_names(quarter_year)
     half_year=decode_monthly_names(half_year)
     # month_ahead['Service Name']=month_ahead['Service Name'].map(Service_Index_to_name)
     # quarter_year['Service Name']=quarter_year['Service Name'].map(Service_Index_to_name)
     # half_year['Service Name']=half_year['Service Name'].map(Service_Index_to_name)
     

     month_ahead['Service Name AR']=month_ahead['Service Name'].map(operations_names_en_to_ar)
     quarter_year['Service Name AR']=quarter_year['Service Name'].map(operations_names_en_to_ar)
     half_year['Service Name AR']=half_year['Service Name'].map(operations_names_en_to_ar)     
     # month_ahead['MAPE']=


     return err,month_MAPE,quarter_year_MAPE, half_year_MAPE, month_ahead,quarter_year, half_year

In [25]:
def daily_prediction_cases(regressor):
     month_ahead = daily_data[(daily_data['Year'] == 2018) & (daily_data['Month'] == 12)] # .sort_values(by=["Count"],ascending=False).iloc[0:10]
     
     quarter_year = daily_data[(daily_data['Year'] == 2018) & (daily_data['Month'] >= 9)]
     
     half_year = daily_data[(daily_data['Year'] == 2018) & (daily_data['Month'] >= 7)]

     month_ahead_pred = regressor.predict(month_ahead.drop('Count', axis=1))
     quarter_year_pred = regressor.predict(quarter_year.drop('Count', axis=1))
     half_year_pred = regressor.predict(half_year.drop('Count', axis=1))
     
     err = mean_absolute_error(month_ahead['Count'], month_ahead_pred)
     
     month_MAPE = mean_absolute_percentage_error(month_ahead['Count'], month_ahead_pred)
     element_month_MAPE = element_mean_absolute_percentage_error(month_ahead['Count'], month_ahead_pred)

     quarter_year_MAPE = mean_absolute_percentage_error( quarter_year['Count'], quarter_year_pred)
     element_quarter_year_MAPE = element_mean_absolute_percentage_error( quarter_year['Count'], quarter_year_pred)

     half_year_MAPE = mean_absolute_percentage_error(half_year['Count'], half_year_pred) 
     element_half_year_MAPE = element_mean_absolute_percentage_error(half_year['Count'], half_year_pred)

     month_ahead["predictions"]=month_ahead_pred
     month_ahead["Month_MAPE"]=element_month_MAPE

     quarter_year["predictions"]=quarter_year_pred
     quarter_year["quarter_MAPE"]= element_quarter_year_MAPE

     half_year["predictions"]=half_year_pred
     half_year["half_year_MAPE"]=element_half_year_MAPE


     month_ahead=decode_monthly_names(month_ahead, uniq)
     quarter_year=decode_monthly_names(quarter_year, uniq)
     half_year=decode_monthly_names(half_year, uniq)
     # month_ahead['Service Name']=month_ahead['Service Name'].map(Service_Index_to_name)
     # quarter_year['Service Name']=quarter_year['Service Name'].map(Service_Index_to_name)
     # half_year['Service Name']=half_year['Service Name'].map(Service_Index_to_name)
     

     month_ahead['Service Name AR']=month_ahead['Service Name'].map(operations_names_en_to_ar)
     quarter_year['Service Name AR']=quarter_year['Service Name'].map(operations_names_en_to_ar)
     half_year['Service Name AR']=half_year['Service Name'].map(operations_names_en_to_ar)     
     # month_ahead['MAPE']=


     return err,month_MAPE,quarter_year_MAPE, half_year_MAPE, month_ahead,quarter_year, half_year

In [67]:
def perform_weekly_regression(regressor):
    # regressor = DecisionTreeRegressor(random_state=0)
    X_train, X_test, y_train, y_test=get_X_y(weekly_processed_data,weekly_split_index)
    regressor.fit(X_train, y_train)
    # y_pred = regressor.predict(X_test)
    # print(f'Total MAE {mean_absolute_percentage_error(y_pred, y_test)}')
    err,week_ahead_MAPE, two_weeks_ahead_MAPE, week_ahead, two_weeks_ahead= weekly_prediction_cases(regressor)

    print(f'1 week predection MAE {err}')
    print(f'1 week predection MAPE {week_ahead_MAPE}')
    print(f'2 weeks predection MAPE {two_weeks_ahead_MAPE}')
    # print(f'half year MAPE {half_year_MAPE}')
    return week_ahead, two_weeks_ahead

In [16]:
def perform_monthly_regression(regressor):
    # regressor = DecisionTreeRegressor(random_state=0)
    X_train, X_test, y_train, y_test=get_X_y(monthly_processed_data,monthly_split_index)
    regressor.fit(X_train, y_train)
    # y_pred = regressor.predict(X_test)
    # print(f'Total MAE {mean_absolute_percentage_error(y_pred, y_test)}')
    err,month_MAPE,quarter_year_MAPE, half_year_MAPE, month_ahead,quarter_year, half_year = monthly_prediction_cases(regressor)

    print(f'1 month predection MAE {err}')
    print(f'1 month predection MAPE {month_MAPE}')
    print(f'Last quarter year MAPE {quarter_year_MAPE}')
    print(f'half year MAPE {half_year_MAPE}')
    return month_ahead,quarter_year, half_year

In [21]:
def perform_daily_regression(regressor):
    # regressor = DecisionTreeRegressor(random_state=0)
    X_train, X_test, y_train, y_test=get_X_y(daily_data,monthly_split_index)
    regressor.fit(X_train, y_train)
    # y_pred = regressor.predict(X_test)
    # print(f'Total MAE {mean_absolute_percentage_error(y_pred, y_test)}')
    err,month_MAPE,quarter_year_MAPE, half_year_MAPE, month_ahead,quarter_year, half_year = daily_prediction_cases(regressor)

    print(f'1 month predection MAE {err}')
    print(f'1 month predection MAPE {month_MAPE}')
    print(f'Last quarter year MAPE {quarter_year_MAPE}')
    print(f'half year MAPE {half_year_MAPE}')
    return month_ahead,quarter_year, half_year

In [18]:
def vis(df):
    fig = go.Figure(data=[
                        go.Bar(name = 'Real Data', x=df["Service Name"], y=df["Count"], marker_color='#800000'),
                        go.Bar(name = 'Predicted', x=df["Service Name"], y=df["predictions"], marker_color='#228B22'),

    ])

    fig.update_layout(barmode='group',width=1200, height=400)
    return fig.show()

1. **Decision Tree Regression**

In [26]:
regressor = DecisionTreeRegressor(random_state=0)
p1 = perform_daily_regression(regressor)

1 month predection MAE 3.3823529411764706
1 month predection MAPE 85.6703332065373
Last quarter year MAPE 82.9380616939965
half year MAPE 86.15206874484237


In [24]:
p1[0]

,Service Name,Count,DayMonth,Month,Year,predictions,Month_MAPE,Service Name AR
11657,Phacoemulsification (Phaco+IOL),48,18,12,2018,41.0,14.583333,فاكو + زرع عدسه
11507,Phacoemulsification (Phaco+IOL),47,5,12,2018,56.0,19.148936,فاكو + زرع عدسه
11480,Phacoemulsification (Phaco+IOL),47,3,12,2018,51.0,8.510638,فاكو + زرع عدسه
11576,Phacoemulsification (Phaco+IOL),44,11,12,2018,14.0,68.181818,فاكو + زرع عدسه
11589,Phacoemulsification (Phaco+IOL),43,12,12,2018,43.0,0.000000,فاكو + زرع عدسه
11468,Phacoemulsification (Phaco+IOL),42,2,12,2018,37.0,11.904762,فاكو + زرع عدسه
11642,Phacoemulsification (Phaco+IOL),41,17,12,2018,43.0,4.878049,فاكو + زرع عدسه
11616,Phacoemulsification (Phaco+IOL),40,15,12,2018,16.0,60.000000,فاكو + زرع عدسه
11495,Phacoemulsification (Phaco+IOL),40,4,12,2018,39.0,2.500000,فاكو + زرع عدسه
11559,Phacoemulsification (Phaco+IOL),38,10,12,2018,23.0,39.473684,فاكو + زرع عدسه


2. **Support Vector Machine Regression**

In [28]:
svr = SVR()
p2 = perform_daily_regression(svr)

1 month predection MAE 4.6010873968289765
1 month predection MAPE 89.59980573907436
Last quarter year MAPE 87.18396885188683
half year MAPE 84.08593429313036


In [29]:
p2[0]

,Service Name,Count,DayMonth,Month,Year,predictions,Month_MAPE,Service Name AR
11444,Athens Protocol,1,1,12,2018,2.081705,108.170483,تثبيت قرنية + PRK
11445,Deep Anterior Lamellar Keratoplasty (DALK),1,1,12,2018,2.678478,167.847833,ترقيع قرنيه
11446,Femto-LASIK,15,1,12,2018,4.364711,70.901925,فيمتو ليزك
11447,Intracorneal Ring Segments (ICRS),1,1,12,2018,2.163314,116.331377,حلقات + تثبيت قرنية
11448,Intravitreal Eylea Injection,5,1,12,2018,2.783171,44.336588,حقن إيليا
...,...,...,...,...,...,...,...,...
11779,Laser-Assisted in Situ Keratomileusis (LASIK),3,30,12,2018,5.709568,90.318943,REDO LASIK
11780,Phacoemulsification (Phaco+IOL),19,30,12,2018,5.263753,72.296038,فاكو + زرع عدسه
11781,Photorefractive Keratomileusis (PRK),1,30,12,2018,1.805793,80.579300,PRK
11782,Removal of Sutures (ROS),1,30,12,2018,1.845417,84.541706,ترقيع قرنيه


3. **AdaBoost Regression**

In [23]:
# ada = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mae'), n_estimators=200, learning_rate=0.02, loss='square',random_state=72)
# x=perform_weekly_regression(ada)
# vis(x[0])

In [31]:
ada = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mae'), n_estimators=300, learning_rate=0.01, loss='square',random_state=7)
x=perform_daily_regression(ada)
# vis(x[0]) 

1 month predection MAE 2.8411764705882354
1 month predection MAPE 67.0230302917265
Last quarter year MAPE 68.06829416207117
half year MAPE 69.64026943018865


In [32]:
x[0]

,Service Name,Count,DayMonth,Month,Year,predictions,Month_MAPE,Service Name AR
11444,Athens Protocol,1,1,12,2018,1.0,0.000000,تثبيت قرنية + PRK
11445,Deep Anterior Lamellar Keratoplasty (DALK),1,1,12,2018,2.0,100.000000,ترقيع قرنيه
11446,Femto-LASIK,15,1,12,2018,9.0,40.000000,فيمتو ليزك
11447,Intracorneal Ring Segments (ICRS),1,1,12,2018,1.0,0.000000,حلقات + تثبيت قرنية
11448,Intravitreal Eylea Injection,5,1,12,2018,3.0,40.000000,حقن إيليا
...,...,...,...,...,...,...,...,...
11779,Laser-Assisted in Situ Keratomileusis (LASIK),3,30,12,2018,2.0,33.333333,REDO LASIK
11780,Phacoemulsification (Phaco+IOL),19,30,12,2018,38.0,100.000000,فاكو + زرع عدسه
11781,Photorefractive Keratomileusis (PRK),1,30,12,2018,1.0,0.000000,PRK
11782,Removal of Sutures (ROS),1,30,12,2018,1.0,0.000000,ترقيع قرنيه


In [38]:
x[0].sort_values(['Count'], ascending=False)[:10]

,Service Name,Count,DayMonth,Month,Year,predictions,Month_MAPE,Service Name AR
11657,Phacoemulsification (Phaco+IOL),48,18,12,2018,41.0,14.583333,فاكو + زرع عدسه
11507,Phacoemulsification (Phaco+IOL),47,5,12,2018,56.0,19.148936,فاكو + زرع عدسه
11480,Phacoemulsification (Phaco+IOL),47,3,12,2018,51.0,8.510638,فاكو + زرع عدسه
11576,Phacoemulsification (Phaco+IOL),44,11,12,2018,14.0,68.181818,فاكو + زرع عدسه
11589,Phacoemulsification (Phaco+IOL),43,12,12,2018,43.0,0.000000,فاكو + زرع عدسه
11468,Phacoemulsification (Phaco+IOL),42,2,12,2018,37.0,11.904762,فاكو + زرع عدسه
11642,Phacoemulsification (Phaco+IOL),41,17,12,2018,43.0,4.878049,فاكو + زرع عدسه
11616,Phacoemulsification (Phaco+IOL),40,15,12,2018,27.0,32.500000,فاكو + زرع عدسه
11495,Phacoemulsification (Phaco+IOL),40,4,12,2018,39.0,2.500000,فاكو + زرع عدسه
11559,Phacoemulsification (Phaco+IOL),38,10,12,2018,23.0,39.473684,فاكو + زرع عدسه


In [25]:
# xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=7)
# perform_monthly_regression(xgb_model)

In [26]:
# gbc=GradientBoostingRegressor(n_estimators=500,max_depth=8,min_samples_split=2,loss="ls",random_state=7)
# perform_monthly_regression(gbc)

** choosing the best Model  till now which is (ADABOOSTING) so We will pickle it **

In [27]:
with open('./ada_regressor.pickle', 'wb') as f:
    pickle.dump(ada, f)

**Testing the pickled model**

In [28]:
with open('./ada_regressor.pickle', 'rb') as f:
  reg = pickle.load(f)
month=perform_monthly_regression(reg)

NameError: name 'monthly_split_index' is not defined

In [29]:
week=perform_weekly_regression(reg)

NameError: name 'weekly_split_index' is not defined

In [30]:
vis(week[1])

NameError: name 'week' is not defined

In [31]:
month[0].head()

NameError: name 'month' is not defined

In [32]:
# week[0].to_csv("pred_one_week_ada.csv",encoding='utf-8-sig')
# month[0].to_csv("pred_one_month_ada.csv",encoding='utf-8-sig')
# quarter.to_csv("pred_last_quarter_ada.csv",encoding='utf-8-sig')
# half_year.to_csv("pred_half_year_ada.csv",encoding='utf-8-sig')